## Workshop script for Session 5 Part 2: Animations of particle trajectories with matplotlib and cartopy.

Data used: CaribbeanCurrent_1994.zarr

About this Parcels dataset: 
Particles are released every 5 days on the transect between Venezuela (mainland) and the island of Grenada.
Particles are advected with the 2D geostrophic flow computed from the Copernicus model output GLORYS12V1 (1/12° horizontal resolution, 50 vertical levels) for the year 1994. In order to decrease the data size for this workshop, outputs are stored every 12 hours.

In this script we will first plot the simplified animation of particle trajectories. Then we will do the following exercises:
1) Create backward animation
2) Start animation at chosen time (e.g., 1st of February 1994)
3) Use different color of particles for every new time of release (every 5 days)
4) (advanced) Add trails to the particles

All solutions can be found in the notebook Session5_animations_tutorial.ipynb

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import timedelta
import matplotlib.cm as cm

In order to see the animation inline in notebook, we need to activate it:

In [2]:
# for interactive display of animation
plt.rcParams["animation.html"] = "jshtml"

Load the data and get familiar with it:

In [ ]:
# Load dataset
ds = xr.open_zarr('data/CaribbeanCurrent_1994.zarr')
print(f"Loaded: {len(ds.trajectory)} particles")

# For performance, load only lon, lat, time into memory
print("Loading subset into memory...")
ds = ds[["lon", "lat", "time"]].load()

ds

We need to set up the time dimension for plotting the particles correctly based on their release times. This particular simulation has an output dt of 12 hours. We set up the **timerange** which determines at which times the particles will start the animation.

In [ ]:
# Setup time dimension:
# For this example our output is stored at every 12 hours
outputdt = timedelta(hours=12)

# Create timerange from min to max time in your dataset
timerange = np.arange(
    np.nanmin(ds["time"].values),
    np.nanmax(ds["time"].values) + np.timedelta64(outputdt),
    outputdt,
)

print(f"Timerange has {len(timerange)} timesteps from {timerange[0]} to {timerange[-1]}")

Following is the code for creating a simple animation, taking into the account the **timerange** and plotting the trajectories as scatter plots.

🐝 **Do the following:**
- if you are new to animations, get familiar with the code
- increase the number of frames to create longer animation

In [ ]:
# Number of timesteps to animate
nframes = 50    # use less frames for testing purposes

# figure setup
fig, ax = plt.subplots(figsize=(6, 5), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_xlim(-71, -59)
ax.set_ylim(9.5, 19.5)
ax.coastlines(color='saddlebrown')
ax.add_feature(cfeature.LAND, alpha=0.5, facecolor='saddlebrown')

# Find particles at the first time step
time_id = np.where(ds["time"] == timerange[0])
initial_lons = ds["lon"].values[time_id]
initial_lats = ds["lat"].values[time_id]

# Remove any NaN values for initial plot
valid_initial = ~np.isnan(initial_lons) & ~np.isnan(initial_lats)
# plot first timestep
scatter = ax.scatter(
    initial_lons[valid_initial], 
    initial_lats[valid_initial], 
    s=2, c='b'
)

# Set initial title
t_str = str(timerange[0])[:19]  # Format datetime nicely
title = ax.set_title(f"Particles at t = {t_str}")

# loop over for animation
def animate(i):
    print(f"Animating frame {i+1}/{len(timerange)} at time {timerange[i]}")
    t_str = str(timerange[i])[:19]
    title.set_text(f"Particles at t = {t_str}")
    
    # Find particles at current time
    time_id = np.where(ds["time"] == timerange[i])
    current_lons = ds["lon"].values[time_id]
    current_lats = ds["lat"].values[time_id]
    
    # Remove NaN values
    valid = ~np.isnan(current_lons) & ~np.isnan(current_lats)
    
    # Update scatter plot positions using scatter.set_offsets
    if np.any(valid):
        scatter.set_offsets(np.c_[current_lons[valid], current_lats[valid]])
    else:
        scatter.set_offsets(np.empty((0, 2)))  # Empty array if no valid particles

# Create animation
anim = matplotlib.animation.FuncAnimation(fig, animate, frames=nframes, interval=100)
anim

## 🐝 **Task 1: Backward animation**

Create animation that starts at the last existing timestep of the Parcels dataset. 

In [ ]:


# Number of timesteps to animate
nframes = 50    # use less frames for testing purposes

# figure setup
fig, ax = plt.subplots(figsize=(6, 5), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_xlim(-71, -59)
ax.set_ylim(9.5, 19.5)
ax.coastlines(color='saddlebrown')
ax.add_feature(cfeature.LAND, alpha=0.5, facecolor='saddlebrown')

... # continue 

## 🐝 **Task 2: Starting animation at a chosen time**

Start animation on the 1st of February 1994.

In [ ]:
# Number of timesteps to animate
nframes = 50    # use less frames for testing purposes

# figure setup
fig, ax = plt.subplots(figsize=(6, 5), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_xlim(-71, -59)
ax.set_ylim(9.5, 19.5)
ax.coastlines(color='saddlebrown')
ax.add_feature(cfeature.LAND, alpha=0.5, facecolor='saddlebrown')

... # continue 

## 🐝 **Task 3: Add colors to particles**

Use different color of particles for every new time of release. 

Extra: think about the colorscheme (in Part 2 Christian will discuss the importance of colors)

In [8]:
# write your code here

## 🐝 **Task 4: Particle trails**

Add particle trails showing the last 10 days of trajectory.

HINT 1: plot trails for every 10th particle to speed up the animation

HINT 2 (optional, see code below): you can pre-compute the data for all particles at all time steps to speed up the animation

```python
all_particles_data = []
for i, target_time in enumerate(timerange):
    time_id = np.where(ds["time"] == target_time)
    lons = ds["lon"].values[time_id]
    lats = ds["lat"].values[time_id]
    particle_indices = time_id[0]
    valid = ~np.isnan(lons) & ~np.isnan(lats)
    all_particles_data.append({
        'lons': lons[valid],
        'lats': lats[valid],
        'particle_indices': particle_indices[valid],
        'valid_count': np.sum(valid)
    })
```

In [ ]:
# write your code here
